In [4]:
import json
from coinbase.rest import RESTClient
from json import dumps
import pandas as pd
from random import randint
import pytz
import datetime
import time

In [5]:
# Get demo API key
def get_demo_key(key_type):
    f = open(f"/home/vikas/Documents/Demo_{key_type}_CG_key.json")
    key_dict = json.load(f)
    return key_dict["name"], key_dict["privateKey"]

In [6]:
api_key, api_secret = get_demo_key("view")

view_client = RESTClient(api_key = api_key,
                         api_secret = api_secret)

## Test API

In [7]:
server_time = view_client.get_unix_time()
print(dumps(server_time, indent = 4))

{
    "iso": "2024-09-08T16:08:16Z",
    "epochSeconds": "1725811696",
    "epochMillis": "1725811696661"
}


In [22]:
trade_client.get_api_key_permissions()

{'can_view': True,
 'can_trade': True,
 'can_transfer': False,
 'portfolio_uuid': '581b97fc-d5a9-4b69-bb3c-1c9853fb9b71',
 'portfolio_type': 'CONSUMER'}

## Convert to local timezone

In [9]:
def convert_to_local_tz(old_ts):
    
    new_tz = pytz.timezone("Europe/Amsterdam")
    old_tz = pytz.timezone("UTC")
    
    format = "%Y-%m-%dT%H:%M:%S.%fZ"
    datetime_obj = datetime.datetime.strptime(old_ts, format)
    
    localized_ts = old_tz.localize(datetime_obj)
    new_ts = localized_ts.astimezone(new_tz)
    
    return new_ts

## Get trades for a given pair

In [10]:
# btc_usd = view_client.get_public_market_trades("BTC-USD", 2)
# print(dumps(btc_usd, indent = 4))

In [11]:
def get_latest_trades(pair, num_trades):
    
    all_trades = view_client.get_public_market_trades(pair, num_trades)
    
    df_trades = pd.DataFrame(all_trades["trades"])
    df_trades["time"] = df_trades["time"].apply(convert_to_local_tz)
    
    return df_trades

In [23]:
df_eth_eur = get_latest_trades("ETH-EUR", 5)
df_eth_eur

,trade_id,product_id,price,size,time,side,bid,ask
0,53371020,ETH-EUR,2032.75,0.02338736,2024-09-08 18:28:55.240611+02:00,SELL,,
1,53371019,ETH-EUR,2032.29,0.00491918,2024-09-08 18:28:49.923723+02:00,BUY,,
2,53371018,ETH-EUR,2032.31,0.00246517,2024-09-08 18:28:49.923723+02:00,BUY,,
3,53371017,ETH-EUR,2032.75,0.09365799,2024-09-08 18:28:32.142804+02:00,SELL,,
4,53371016,ETH-EUR,2032.53,0.0043712,2024-09-08 18:28:30.192674+02:00,BUY,,


## Get accounts

In [13]:
#client.get_accounts()

## Get portfolios

In [24]:
trade_client.get_portfolios()

{'portfolios': [{'name': 'Default',
   'uuid': 'd083ea15-3f44-54da-8e9e-2a5a9bdccb32',
   'type': 'DEFAULT',
   'deleted': False},
  {'name': 'Demo portfolio',
   'uuid': '581b97fc-d5a9-4b69-bb3c-1c9853fb9b71',
   'type': 'CONSUMER',
   'deleted': False}]}

## Check portfolio breakdown

In [15]:
api_key, api_secret = get_demo_key("trade")

trade_client = RESTClient(api_key = api_key,
                          api_secret = api_secret)

In [16]:
trade_client.get_portfolio_breakdown('581b97fc-d5a9-4b69-bb3c-1c9853fb9b71')

{'breakdown': {'portfolio': {'name': 'Demo portfolio',
   'uuid': '581b97fc-d5a9-4b69-bb3c-1c9853fb9b71',
   'type': 'CONSUMER',
   'deleted': False},
  'portfolio_balances': {'total_balance': {'value': '19.4', 'currency': 'EUR'},
   'total_futures_balance': {'value': '0', 'currency': 'EUR'},
   'total_cash_equivalent_balance': {'value': '15', 'currency': 'EUR'},
   'total_crypto_balance': {'value': '4.4', 'currency': 'EUR'},
   'futures_unrealized_pnl': {'value': '0', 'currency': 'EUR'},
   'perp_unrealized_pnl': {'value': '0', 'currency': 'EUR'}},
  'spot_positions': [{'asset': 'EUR',
    'account_uuid': '250a2cb1-2aa3-5b0b-bb56-a17cbc2f9780',
    'total_balance_fiat': 15,
    'total_balance_crypto': 15,
    'available_to_trade_fiat': 15,
    'allocation': 0.7729604,
    'cost_basis': {'value': '15', 'currency': 'EUR'},
    'asset_img_url': '',
    'is_cash': True,
    'average_entry_price': {'value': '1.00', 'currency': 'EUR'},
    'asset_uuid': '',
    'available_to_trade_crypto': 

## Place a market buy order

In [25]:
def generate_client_id(id_length):
    
    return "".join(f"{randint(0, 9)}" for _ in range(id_length))       

In [30]:
# buy_order = trade_client.market_order_buy(
#     client_order_id = generate_client_id(10),
#     product_id = "ETH-EUR",
#     quote_size = "5")   

In [28]:
buy_order

{'success': True,
 'success_response': {'order_id': 'b870a89a-c5fa-4b85-abfe-42d929ee8a39',
  'product_id': 'ETH-EUR',
  'side': 'BUY',
  'client_order_id': '6302915579',
  'attached_order_id': ''},
 'order_configuration': {'market_market_ioc': {'quote_size': '5'}}}

In [20]:
# fills = trade_client.get_fills(order_id = buy_order["order_id"])
# fills

In [31]:
#trade_client.get_order('b870a89a-c5fa-4b85-abfe-42d929ee8a39')

In [33]:
buy_order_preview = trade_client.preview_market_order_buy(
                    product_id = "ETH-EUR",
                    quote_size = "5")   

In [35]:
#buy_order_preview

## Place a market sell order